In [ ]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/Dominik/R-workspace/cecelia/inst')

# MacOS
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [ ]:
base_dir = '/Volumes/USER_data/Dominik/CECELIA_BACKUP/CV5iNI/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/HYmMxh/')
im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'HYmMxh')

In [ ]:
import py.zarr_utils as zarr_utils

im, _ = zarr_utils.open_as_zarr(im_path)
im = im[0]

In [ ]:
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils

# get OME-XML
omexml = ome_xml_utils.parse_meta(im_path)

# create dim utils for image
dim_utils = DimUtils(omexml, use_channel_axis = True)
dim_utils.calc_image_dimensions(im.shape)

In [ ]:
import torch

In [ ]:
img_one = None
img_one_tensor = torch.zeros(61, dim_utils.dim_val('X'), dim_utils.dim_val('Y'))

In [ ]:
memmap_image = tifffile.memmap(
    f"{self.parent.save_header}/{metadata}/denoised.tif",
    shape = self.parent.img_shape,
    dtype="float32"
)

with torch.no_grad():
    (t, _, _) = self.parent.img_shape
    self.parent.model.eval()

    # for ti in range(t):
    for ti in range(
        self.parent.actual_start_idx - 1, self.parent.actual_end_idx
    ):
        if self._isRunning == False:
            break
        self._ti = ti

        counter += 1
        if (time.time() - start_time) > 1:
            fps = counter / (time.time() - start_time)
            self.parent.Label_fps.setText(f"fps : {fps:2.2f}")
            counter = 0
            start_time = time.time()

        start_idx = max(ti - 30, 0)
        end_idx = min(ti + 31, t)
        # print(ti)

        pad_front = max(30 - ti, 0)
        pad_end = max(31 - (t - ti), 0)
        # print(pad_front, pad_end)

        img_one = tifffile.imread(
            self.parent.img_path, key=range(start_idx, end_idx, 1)
        )
        img_one_tensor[pad_front : 61 - pad_end, :, :] = torch.from_numpy(
            img_one.astype(np.float32)
        ).type(torch.FloatTensor)
        if pad_front >= 1:
            img_one_tensor[0:pad_front, :, :] = img_one_tensor[
                pad_front
            ].repeat(pad_front, 1, 1)
        if pad_end >= 1:
            img_one_tensor[61 - pad_end :, :, :] = img_one_tensor[
                61 - pad_end - 1
            ].repeat(pad_end, 1, 1)

        if ti == self.parent.actual_start_idx - 1:
            mean_one = torch.mean(img_one_tensor).item()
            std_one = torch.std(img_one_tensor).item()

        w = 128 if self.parent.img_w > 128 else self.parent.img_w
        h = 128 if self.parent.img_h > 128 else self.parent.img_h

        testset = DatasetSupport_test_stitch(
            img_one_tensor.clone().detach(),
            patch_size=[61, w, h],
            patch_interval=[1, w // 2, h // 2],
            mean_image=mean_one,
            std_image=std_one,
        )
        testloader = torch.utils.data.DataLoader(testset, batch_size=4)
        self.test_dataloader = testloader

        denoised_frame = np.zeros(
            (1, *self.test_dataloader.dataset.noisy_image.shape[1:]),
            dtype=np.float32,
        )

        for _, (noisy_image, _, single_coordinate) in enumerate(
            self.test_dataloader
        ):
            if self.parent.cuda:
                noisy_image = noisy_image.cuda()  # [b, z, y, x]

            noisy_image_denoised = self.parent.model(noisy_image)

            for bi in range(noisy_image.size(0)):
                stack_start_w = int(single_coordinate["stack_start_w"][bi])
                stack_end_w = int(single_coordinate["stack_end_w"][bi])
                patch_start_w = int(single_coordinate["patch_start_w"][bi])
                patch_end_w = int(single_coordinate["patch_end_w"][bi])

                stack_start_h = int(single_coordinate["stack_start_h"][bi])
                stack_end_h = int(single_coordinate["stack_end_h"][bi])
                patch_start_h = int(single_coordinate["patch_start_h"][bi])
                patch_end_h = int(single_coordinate["patch_end_h"][bi])


                denoised_frame[
                    0,  # stack_start_s,
                    stack_start_h:stack_end_h,
                    stack_start_w:stack_end_w,
                ] = (
                    noisy_image_denoised[bi]
                    .squeeze()[
                        patch_start_h:patch_end_h, patch_start_w:patch_end_w
                    ]
                    .cpu()
                )

        denoised_frame = denoised_frame * std_one + mean_one

        disp_raw = img_one_tensor[30].detach().numpy()
        disp_denoised = denoised_frame[0]

        if ti == self.parent.actual_start_idx - 1:
            vmin = np.min(disp_raw)
            vmax = np.percentile(disp_raw, q=99) # np.max(disp_raw)

        self.parent.disp_raw = np.clip(
            1.2 * (disp_raw - vmin) / (vmax - vmin), 0, 1
        )
        self.parent.disp_denoised = np.clip(
            1.2 * (disp_denoised - vmin) / (vmax - vmin), 0, 1
        )

        self.signal_update_img.emit(1)
        self.progressbar_update.emit(1)

        memmap_image[ti, :, :] = denoised_frame

        # if ti == self.parent.actual_start_idx - 1:
            # tifffile.imwrite(
            #     f"{self.parent.save_header}/{metadata}/denoised.tif",
            #     denoised_frame,
            #     dtype="float32",
            #     metadata={'axes': 'TYX', 'imagej_metadata': self.parent.imagej_metadata, }
            # )
        # else:
            # tifffile.imwrite(
            #     f"{self.parent.save_header}/{metadata}/denoised.tif",
            #     denoised_frame,
            #     append=True,
            #     dtype="float32",
            #     metadata={'axes': 'TYX', 'imagej_metadata': self.parent.imagej_metadata}
            # )

    memmap_image.flush()
    del memmap_image